In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

In [4]:
# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

In [5]:
print(len(feature_files))

1802


In [6]:
# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

In [7]:
# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(['Arousal_Value', 'Valence_Value', 'frameTime'], axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

In [8]:
# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

In [20]:
# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_v = []
mae_v = []
rmse_v = []
r2_v = []
f1_v = []
accuracy_v = []
precision_v = []
recall_v = []

In [21]:
%%time

n_fold=0

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):

    n_fold=n_fold+1
    print("fold : ",n_fold)


    X_train, X_test = X[train_index], X[test_index]
    y_v_train, y_v_test = y_v[train_index], y_v[test_index]


    # Initialize and train the SVR model
    lr_model_valence= LinearRegression()

    lr_model_valence.fit(X_train, y_v_train)

    # Evaluate the model on the test set
    predictions_v = lr_model_valence.predict(X_test)


    # Calculate additional evaluation metrics
    mse_v.append(mean_squared_error(y_v_test, predictions_v))
    mae_v.append(mean_absolute_error(y_v_test, predictions_v))
    rmse_v.append(np.sqrt(mean_squared_error(y_v_test, predictions_v)))
    r2_v.append(r2_score(y_v_test, predictions_v))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_v = (predictions_v >= threshold).astype(int)
    y_v_test_binary = (y_v_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_v.append(f1_score(y_v_test_binary, binary_predictions_v))
    accuracy_v.append(accuracy_score(y_v_test_binary, binary_predictions_v))
    precision_v.append(precision_score(y_v_test_binary, binary_predictions_v))
    recall_v.append(recall_score(y_v_test_binary, binary_predictions_v))




fold :  1
fold :  2
fold :  3
fold :  4
fold :  5
CPU times: user 14.9 s, sys: 2.66 s, total: 17.6 s
Wall time: 12.1 s


In [22]:
# Calculate average performance across all folds
average_mse_v = np.mean(mse_v)
average_mae_v = np.mean(mae_v)
average_rmse_v = np.mean(rmse_v)
average_r2_v = np.mean(r2_v)
average_f1_v = np.mean(f1_v)
average_accuracy_v = np.mean(accuracy_v)
average_precision_v = np.mean(precision_v)
average_recall_v = np.mean(recall_v)

print(f'Average Arousal MSE: {average_mse_v:.4f}')
print(f'Average Arousal MAE: {average_mae_v:.4f}')
print(f'Average Arousal RMSE: {average_rmse_v:.4f}')
print(f'Average Arousal R2 Score: {average_r2_v:.4f}')
print(f'Average Arousal F1 Score: {average_f1_v:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_v:.4f}')
print(f'Average Arousal Precision: {average_precision_v:.4f}')
print(f'Average Arousal Recall: {average_recall_v:.4f}')

Average Arousal MSE: 0.0377
Average Arousal MAE: 0.1554
Average Arousal RMSE: 0.1942
Average Arousal R2 Score: 0.3179
Average Arousal F1 Score: 0.8164
Average Arousal Accuracy: 0.7396
Average Arousal Precision: 0.7624
Average Arousal Recall: 0.8785
